# Pandas resample() tricks you should know for manipulating time-series data
Time-series data is common in data science projects. Often, you may be interested in resampling your time-series data into the frequency that you want to analyze data or draw additional insights from data [1].

In this article, we’ll be going through some examples of resampling time-series data using Pandas resample() function. We will cover the following common problems and should help you get started with time-series data manipulation.

*  Downsampling and performing aggregation
*    Downsampling with a custom base
*    Upsampling and filling values
*    A practical example

## Downsampling and performing aggregation

Downsampling is to resample a time-series dataset to a wider time frame. For example, from minutes to hours, from days to years. The result will have a reduced number of rows and values can be aggregated with mean(), min(), max(), sum() etc.

Let’s see how it works with the help of an example.

Suppose we have a dataset about sales.

In [1]:
import pandas as pd 
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df = pd.read_csv(
    './data/sales_data.csv')

df.head()

,date,num_sold
0,2017-01-02 09:02:03,5
1,2017-01-02 09:14:13,7
2,2017-01-02 09:21:00,5
3,2017-01-02 09:28:57,9
4,2017-01-02 09:42:14,1


In [2]:
df_sales = pd.read_csv(
    './data/sales_data.csv', 
    parse_dates=['date'], 
    index_col=['date']
)

df_sales.head()

,num_sold
date,
2017-01-02 09:02:03,5
2017-01-02 09:14:13,7
2017-01-02 09:21:00,5
2017-01-02 09:28:57,9
2017-01-02 09:42:14,1


To get the total number of sales added every 2 hours, we can simply use resample() to downsample the DataFrame into 2-hour bins and sum the values of the timestamps falling into a bin.

In [3]:
df_sales.resample('2H').sum()

,num_sold
date,
2017-01-02 08:00:00,37
2017-01-02 10:00:00,66
2017-01-02 12:00:00,81
2017-01-02 14:00:00,50
2017-01-02 16:00:00,64
2017-01-02 18:00:00,66
2017-01-02 20:00:00,44
2017-01-02 22:00:00,45


To perform multiple aggregations, we can pass a list of aggregation functions to agg() method.

In [4]:
df_sales.resample('2H').agg(['min','max', 'sum'])

num_sold        
                         min max sum
date                                
2017-01-02 08:00:00        1   9  37
2017-01-02 10:00:00        1   9  66
2017-01-02 12:00:00        1   9  81
2017-01-02 14:00:00        1   9  50
2017-01-02 16:00:00        1   8  64
2017-01-02 18:00:00        1   9  66
2017-01-02 20:00:00        1   9  44
2017-01-02 22:00:00        2   6  45

## Downsampling with a custom base

By default, for the frequencies that evenly subdivide 1 day/month/year, the “origin” of the aggregated intervals is defaulted to `0`. So, for the 2H frequency, the result range will be `00:00:00`, `02:00:00`, `04:00:00`, …, `22:00:00`.

For the sales data we are using, the first record has a date value 2017–01–02 09:02:03 , so it makes much more sense to have the output range start with 09:00:00, rather than 08:00:00. To do that, we can set the “origin” of the aggregated intervals to a different value using the argument base, for example, set base=1 so the result range can start with 09:00:00.


![](./i/1oEJpC3wudNTyZSE-rimJKw.png)

In [5]:
df_sales.resample('2H', base=1).sum()

C:\Users\nemat\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'base' in .resample() and in Grouper() is deprecated.
The new arguments that you should use are 'offset' or 'origin'.

>>> df.resample(freq="3s", base=2)

becomes:

>>> df.resample(freq="3s", offset="2s")

  """Entry point for launching an IPython kernel.


,num_sold
date,
2017-01-02 09:00:00,62
2017-01-02 11:00:00,77
2017-01-02 13:00:00,64
2017-01-02 15:00:00,55
2017-01-02 17:00:00,72
2017-01-02 19:00:00,48
2017-01-02 21:00:00,70
2017-01-02 23:00:00,5


## Upsampling and filling values

Upsampling is the opposite operation of downsampling. It resamples a time-series dataset to a smaller time frame. For example, from hours to minutes, from years to days. The result will have an increased number of rows and additional rows values are defaulted to NaN. The built-in method ffill() and bfill() are commonly used to perform forward filling or backward filling to replace NaN.

Let’s make up a DataFrame for demonstration.

In [6]:
df = pd.DataFrame(
    { 'value': [1, 2, 3] }, 
    index=pd.period_range(
        '2012-01-01',
         freq='A',
         periods=3
    )
)
df

,value
2012,1
2013,2
2014,3


To resample a year by quarter and forward filling the values. The forward fill method ffill() will use the last known value to replace NaN.
![](./i/1_PHEUoOLiCtJe5KtCKnO5hg.png)

In [7]:
df.resample('Q').ffill()

,value
2012Q1,1
2012Q2,1
2012Q3,1
2012Q4,1
2013Q1,2
2013Q2,2
2013Q3,2
2013Q4,2
2014Q1,3
2014Q2,3


To resample a year by quarter and backward filling the values. The backward fill method bfill() will use the next known value to replace NaN.

![](./i/1_LXPqyzf-QAYye4YCJ6fZcw.png)

In [8]:
df.resample('Q').bfill()

,value
2012Q1,1.0
2012Q2,2.0
2012Q3,2.0
2012Q4,2.0
2013Q1,2.0
2013Q2,3.0
2013Q3,3.0
2013Q4,3.0
2014Q1,3.0
2014Q2,NaN


## A Practical example

Let’s take a look at how to use Pandas resample() to deal with a real-world problem.

Suppose we have 2 datasets, one for monthly sales df_sales and the other for price df_price. The df_price only has records on price changes.

### Step 1: Resample price dataset by month and forward fill the values

In [9]:
# load sales
df_sales = pd.read_csv('data/sales.csv', parse_dates=['date'], index_col=['date'])
df_sales.head()

,num_sold
date,
2018-01-31,5
2018-02-28,17
2018-03-31,5
2018-04-30,16
2018-05-31,12


In [10]:
# load price
df_price = pd.read_csv('data/price.csv', parse_dates=['date'], index_col=['date'])
df_price.head()

,price
date,
2018-01-31,16.0
2018-05-31,15.5
2018-12-31,10.0


In [11]:
df_price = df_price.resample('M').ffill()
df_price

,price
date,
2018-01-31,16.0
2018-02-28,16.0
2018-03-31,16.0
2018-04-30,16.0
2018-05-31,15.5
2018-06-30,15.5
2018-07-31,15.5
2018-08-31,15.5
2018-09-30,15.5


![](./i/1_3yuXIWOGIhUlrsl8NwhgMw.png)


### Step 2: Combine results and calculate total sales

In [12]:
df = pd.concat([df_sales, df_price], axis = 1)
df

,num_sold,price
date,,
2018-01-31,5,16.0
2018-02-28,17,16.0
2018-03-31,5,16.0
2018-04-30,16,16.0
2018-05-31,12,15.5
2018-06-30,12,15.5
2018-07-31,2,15.5
2018-08-31,9,15.5
2018-09-30,5,15.5


Pandas concat() function with argument axis=1 is used to combine df_sales and df_price horizontally. 

After that, the total sales can be calculated using the element-wise multiplication df['num_sold'] * df['price'].

By executing the above statement, you should get an output like below:
    
[](./i/AhPi27cPdze67s13m4s2bw.png)

In [13]:
df['total_sales'] = df['num_sold'] * df['price']
df

,num_sold,price,total_sales
date,,,
2018-01-31,5,16.0,80.0
2018-02-28,17,16.0,272.0
2018-03-31,5,16.0,80.0
2018-04-30,16,16.0,256.0
2018-05-31,12,15.5,186.0
2018-06-30,12,15.5,186.0
2018-07-31,2,15.5,31.0
2018-08-31,9,15.5,139.5
2018-09-30,5,15.5,77.5


# All the Pandas shift() you should know for data analysis

Suppose you encountered a situation where you need to push all rows in a DataFrame or require to use the previous row in a DataFrame. Maybe you want to calculate the difference in consecutive rows, Pandas shift() would be an ideal way to achieve these objectives.

In this article, we’ll be going through some examples of manipulating data using Pandas shift() function. We will focus on practical problems and should help you get started with data analysis.

1. Shifting values with periods
2. Shifting time-series data with freq
3. A practical example: calculating the difference in consecutive rows
4. A practical example: calculating the 7 days difference for time-series data

## Shifting values with periods

Pandas shift() shift index by the desired number of periods. The simplest call should have an argument periods (It defaults to 1) and it represents the number of shifts for the desired axis. And by default, it is shifting values vertically along the axis 0 . NaN will be filled for missing values introduced as a result of the shifting.

Let’s see how this works with the help of an example.

![](./i/lGPAbHRtK1TArAzB_6hzVQ.png)

In [14]:
df = pd.DataFrame({
    "A": [1, 2, 3, 4, 5],
    "B": [10, 20, 30, 40, 50]
})

df

,A,B
0,1,10
1,2,20
2,3,30
3,4,40
4,5,50


In [15]:
df.shift(periods=1)
df.shift(1)

,A,B
0,NaN,NaN
1,1.0,10.0
2,2.0,20.0
3,3.0,30.0
4,4.0,40.0


,A,B
0,NaN,NaN
1,1.0,10.0
2,2.0,20.0
3,3.0,30.0
4,4.0,40.0


To replace NaN , you can use the argument fill_value , for example, replace NaN with 0

In [16]:
df.shift(periods=1 , fill_value=0)

,A,B
0,0,0
1,1,10
2,2,20
3,3,30
4,4,40


In [17]:
# In addition, you can pass a negative number to periods and it will shift values in the oppositive direction.
df.shift(periods=-1 )

,A,B
0,2.0,20.0
1,3.0,30.0
2,4.0,40.0
3,5.0,50.0
4,NaN,NaN


In [18]:
# To shift values horizontally, you can set axis=1
df.shift(periods=1, axis=1)

,A,B
0,NaN,1
1,NaN,2
2,NaN,3
3,NaN,4
4,NaN,5


## Shifting time-series data with freq

Pandas shift() function has an argument called freq which allows you to do the frequency-based shifting. The Pandas shift() function is beneficial when dealing with time-series data.

In order to use the argument freq, you need to make sure the index of DataFrame is date or datetime, otherwise, it will raise a NotImplementedError.

Let’s see how this works with the help of an example.

In [19]:
df = pd.DataFrame({
        "A": [1, 2, 3, 4, 5],
        "B": [10, 20, 30, 40, 50]
    },  
    index=pd.date_range("2020-01-01", freq='D', periods=5)
)
df
df.shift(freq='10D')# The equivalent
df.shift(periods=10, freq='D')
df.shift(periods=10)

,A,B
2020-01-01,1,10
2020-01-02,2,20
2020-01-03,3,30
2020-01-04,4,40
2020-01-05,5,50


,A,B
2020-01-11,1,10
2020-01-12,2,20
2020-01-13,3,30
2020-01-14,4,40
2020-01-15,5,50


,A,B
2020-01-11,1,10
2020-01-12,2,20
2020-01-13,3,30
2020-01-14,4,40
2020-01-15,5,50


,A,B
2020-01-01,NaN,NaN
2020-01-02,NaN,NaN
2020-01-03,NaN,NaN
2020-01-04,NaN,NaN
2020-01-05,NaN,NaN


## A practical example: calculating the difference in consecutive rows

Suppose you need to use the previous row value to calculate the sales change, Pandas shift() would be a way to achieve this task.

In [20]:
df = pd.DataFrame({
    "date": pd.date_range("2020-01-01", freq='D', periods=5),
    "sales": [22, 30, 32, 25, 42]
})
df
df['shift_sales'] = df.shift(1)['sales']
# To calculate the sales change in consecutive rows.
df['diff'] = df['sales'] - df.shift(1)['sales']
df

,date,sales
0,2020-01-01,22
1,2020-01-02,30
2,2020-01-03,32
3,2020-01-04,25
4,2020-01-05,42


,date,sales,shift_sales,diff
0,2020-01-01,22,NaN,NaN
1,2020-01-02,30,22.0,8.0
2,2020-01-03,32,30.0,2.0
3,2020-01-04,25,32.0,-7.0
4,2020-01-05,42,25.0,17.0


## A practical example: calculating the 7 days difference for time-series data

Now, suppose you have been asked to calculate the 7 days sales change as follows

value_1 = Day_8 - Day_1
value_2 = Day_9 - Day_2
value_3 = Day_10 - Day_3
...
value_n = Day_N - Day_N-7

Pandas shift() with the argument freq would be an ideal way to achieve this task. Let’s use read_csv() with the argument parse_dates and index_col to load data into a DataFrame.

![](./i/1BJdB-9vrh5rGeO75-d_pwQ.png)

In [21]:
df = pd.read_csv('./data/time_series.csv' , parse_dates=['date'] , index_col=['date'])
df

the_7_days_diff = df['sales'] - df.shift(freq='7D')['sales']
the_7_days_diff

,sales
date,
2020-01-01,22
2020-01-02,30
2020-01-03,32
2020-01-04,25
2020-01-05,42
2020-01-06,20
2020-01-07,45
2020-01-09,43
2020-01-10,27


date
2020-01-01     NaN
2020-01-02     NaN
2020-01-03     NaN
2020-01-04     NaN
2020-01-05     NaN
2020-01-06     NaN
2020-01-07     NaN
2020-01-08     NaN
2020-01-09    13.0
2020-01-10    -5.0
2020-01-11     NaN
2020-01-12     NaN
2020-01-13     NaN
2020-01-14     NaN
2020-01-16     NaN
2020-01-17     NaN
Name: sales, dtype: float64

### Notice that

    There is a record for “2020–01–08” in the result
    The value of “2020–01–08” is NaN because df doesn’t have this value
    Values for the date from “2020–01–01” to “2020–01–07” are NaN. This is because df.shift(freq='7D') doesn’t have these values.
    The last 6 records are NaN because df doesn’t have these values

# Pandas convert JSON into a DataFrame
DataFrame and Series are two core data structures in Pandas. DataFrame is a 2-dimensional labeled data with rows and columns. It is like a spreadsheet or SQL table. Series is a 1-dimensional labeled array. It is sort of like a more powerful version of the Python list. Understanding Series is very important, not only because it is one of the core data structures, but also because it is the building blocks of a DataFrame.

In this article, you’ll learn the most commonly used data operations with Pandas Series and should help you get started with Pandas. The article is structured as follows:

    Creating a Series
    Retrieving elements
    Attributes (commonly used)
    Methods (commonly used)
    Working with Python built-in functions
    
## Creating a Series

In [22]:
df = pd.read_json('./data/simple.json')
df
df.info()
df.describe()

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         3 non-null      object
 1   name       3 non-null      object
 2   math       3 non-null      int64 
 3   physics    3 non-null      int64 
 4   chemistry  3 non-null      int64 
dtypes: int64(3), object(2)
memory usage: 248.0+ bytes


,math,physics,chemistry
count,3.00000,3.000000,3.000000
mean,76.00000,77.333333,63.333333
std,14.73092,12.055428,13.650397
min,60.00000,66.000000,51.000000
25%,69.50000,71.000000,56.000000
50%,79.00000,76.000000,61.000000
75%,84.00000,83.000000,69.500000
max,89.00000,90.000000,78.000000


## read from html

In [23]:
URL = 'http://raw.githubusercontent.com/BindiChen/machine-learning/master/data-analysis/027-pandas-convert-json/data/simple.json'
df = pd.read_json(URL)
df

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78


##  Flattening nested list from JSON object

Pandas read_json() works great for flattened JSON like we have in the previous example. What about JSON with a nested list? Let’s see how to convert the following JSON into a DataFrame:

In [24]:
df = pd.read_json('data/nested_list.json')
df

import json
# load data using Python JSON module
with open('data/nested_list.json','r') as f:
    data = json.loads(f.read())
    
    
df_nested_list = pd.json_normalize(data, record_path =['students'])
df_nested_list

# To include school_name and class
df_nested_list = pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['school_name', 'class']
)

df_nested_list

,school_name,class,students
0,ABC primary school,Year 1,"{'id': 'A001', 'name': 'Tom', 'math': 60, 'phy..."
1,ABC primary school,Year 1,"{'id': 'A002', 'name': 'James', 'math': 89, 'p..."
2,ABC primary school,Year 1,"{'id': 'A003', 'name': 'Jenny', 'math': 79, 'p..."


,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78


,id,name,math,physics,chemistry,school_name,class
0,A001,Tom,60,66,61,ABC primary school,Year 1
1,A002,James,89,76,51,ABC primary school,Year 1
2,A003,Jenny,79,90,78,ABC primary school,Year 1


## Flattening nested list and dict from JSON object

Next, let’s try to read a more complex JSON data, with a nested list and a nested dictionary.

In [25]:

# load data using Python JSON module
with open('data/nested_mix.json','r') as f:
    data = json.loads(f.read())
    
# Normalizing data
df = pd.json_normalize(data, record_path =['students'])

df

df = pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=[
        'class',
        ['info', 'president'], 
        ['info', 'contacts', 'tel']
    ]
)

df

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78


,id,name,math,physics,chemistry,class,info.president,info.contacts.tel
0,A001,Tom,60,66,61,Year 1,John Kasich,123456789
1,A002,James,89,76,51,Year 1,John Kasich,123456789
2,A003,Jenny,79,90,78,Year 1,John Kasich,123456789


## Extracting a single value from deeply nested JSON

Pandas json_normalize() can do most of the work when working with nested data from a JSON file. However, it flattens the entire nested data when your goal might actually be to extract one value. For example, to extract the property math from the following JSON file.

In [28]:
# pip install glom

In [27]:
from glom import glom
df = pd.read_json('data/nested_deep.json')
df['students'].apply(lambda row: glom(row, 'grade.math'))

0    60
1    89
2    79
Name: students, dtype: int64

https://towardsdatascience.com/all-pandas-json-normalize-you-should-know-for-flattening-json-13eae1dfb7dd
    
https://github.com/BindiChen/machine-learning/blob/master/data-analysis/028-pandas-json_normalize/pandas-json_normalize.ipynb

# Pandas json_normalize()
## Flattening a simple JSON

In [29]:
a_dict = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
}

In [31]:
df = pd.json_normalize(a_dict)
df

,school,location,ranking
0,ABC primary school,London,2


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   school    1 non-null      object
 1   location  1 non-null      object
 2   ranking   1 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 152.0+ bytes


## A list of dicts

In [35]:
json_list = [
    { 'class': 'Year 1', 'num_of_students': 20, 'room': 'Yellow' },
    { 'class': 'Year 2', 'num_of_students': 25, 'room': 'Blue' },
]
pd.json_normalize(json_list)

,class,num_of_students,room
0,Year 1,20,Yellow
1,Year 2,25,Blue


In [39]:
json_list = [
    { 'class': 'Year 1', 'num_of_students': 20, 'room': 'Yellow' },
    { 'class': 'Year 2', 'room': 'Blue' }, # no num_of_students
]

pd.json_normalize(json_list)

# We can see that no error is thrown and those missing keys are shown as NaN.

,class,num_of_students,room
0,Year 1,20.0,Yellow
1,Year 2,NaN,Blue


## Flattening a JSON with Multiple levels¶
### A dict

In [40]:
a_obj = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
    'info': {
        'president': 'John Kasich',
        'contacts': {
          'email': {
              'admission': 'admission@abc.com',
              'general': 'info@abc.com'
          },
          'tel': '123456789',
      }
    }
}

pd.json_normalize(a_obj)
# If you don’t want to dig all the way down into each sub-object use the max_level argument.
pd.json_normalize(a_obj, max_level=1)

,school,location,ranking,info.president,info.contacts.email.admission,info.contacts.email.general,info.contacts.tel
0,ABC primary school,London,2,John Kasich,admission@abc.com,info@abc.com,123456789


,school,location,ranking,info.president,info.contacts
0,ABC primary school,London,2,John Kasich,"{'email': {'admission': 'admission@abc.com', '..."


In [42]:
json_list = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask' 
            }
        }
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'math': 'Alan Turing', 
                'physics': 'Albert Einstein' 
            }
        }
    },
]

pd.json_normalize(json_list)

pd.json_normalize(json_list, max_level=1)

,class,student count,room,info.teachers.math,info.teachers.physics
0,Year 1,20,Yellow,Rick Scott,Elon Mask
1,Year 2,25,Blue,Alan Turing,Albert Einstein


,class,student count,room,info.teachers
0,Year 1,20,Yellow,"{'math': 'Rick Scott', 'physics': 'Elon Mask'}"
1,Year 2,25,Blue,"{'math': 'Alan Turing', 'physics': 'Albert Ein..."


##  Flattening JSON with a nested list

What about JSON with a nested list?
When the data is a dict

Let’s see how to flatten the following JSON into a DataFrame:

In [45]:
a_dict = {
    'school': 'ABC primary school',
    'location': 'London',
    'ranking': 2,
    'info': {
        'president': 'John Kasich',
        'contacts': {
          'email': {
              'admission': 'admission@abc.com',
              'general': 'info@abc.com'
          },
          'tel': '123456789',
      }
    },
    'students': [
      { 'name': 'Tom' },
      { 'name': 'James' },
      { 'name': 'Jacqueline' }
    ],
}

# Notes the value of students is a nested list. By calling pd.json_normalize(json_obj), we get:

pd.json_normalize(a_dict)

# Flatten students
pd.json_normalize(a_dict, record_path =['students'])

,school,location,ranking,students,info.president,info.contacts.email.admission,info.contacts.email.general,info.contacts.tel
0,ABC primary school,London,2,"[{'name': 'Tom'}, {'name': 'James'}, {'name': ...",John Kasich,admission@abc.com,info@abc.com,123456789


,name
0,Tom
1,James
2,Jacqueline


In [47]:
# Include meta data
pd.json_normalize(
    a_dict, 
    record_path =['students'],
    meta=['school', ['info', 'contacts', 'tel']],
)

,name,school,info.contacts.tel
0,Tom,ABC primary school,123456789
1,James,ABC primary school,123456789
2,Jacqueline,ABC primary school,123456789


## A list of dicts

In [50]:
json_list = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask' 
            }
        },
        'students': [
            { 
                'name': 'Tom', 
                'sex': 'M', 
                'grades': { 'math': 66, 'physics': 77 } 
            },
            { 
                'name': 'James', 
                'sex': 'M', 
                'grades': { 'math': 80, 'physics': 78 } 
            },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'math': 'Alan Turing', 
                'physics': 'Albert Einstein' 
            }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

pd.json_normalize(json_list)

pd.json_normalize(json_list, record_path =['students'])


pd.json_normalize(
    json_list, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']]
)

,class,student count,room,students,info.teachers.math,info.teachers.physics
0,Year 1,20,Yellow,"[{'name': 'Tom', 'sex': 'M', 'grades': {'math'...",Rick Scott,Elon Mask
1,Year 2,25,Blue,"[{'name': 'Tony', 'sex': 'M'}, {'name': 'Jacqu...",Alan Turing,Albert Einstein


,name,sex,grades.math,grades.physics
0,Tom,M,66.0,77.0
1,James,M,80.0,78.0
2,Tony,M,NaN,NaN
3,Jacqueline,F,NaN,NaN


,name,sex,grades.math,grades.physics,class,room,info.teachers.math
0,Tom,M,66.0,77.0,Year 1,Yellow,Rick Scott
1,James,M,80.0,78.0,Year 1,Yellow,Rick Scott
2,Tony,M,NaN,NaN,Year 2,Blue,Alan Turing
3,Jacqueline,F,NaN,NaN,Year 2,Blue,Alan Turing


## Ignoring KeyError if keys are not always present

In [55]:
data = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Elon Mask',
            }
        },
        'students': [
            { 'name': 'Tom', 'sex': 'M' },
            { 'name': 'James', 'sex': 'M' },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'physics': 'Albert Einstein'
            }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

data

[{'class': 'Year 1',
  'student count': 20,
  'room': 'Yellow',
  'info': {'teachers': {'math': 'Rick Scott', 'physics': 'Elon Mask'}},
  'students': [{'name': 'Tom', 'sex': 'M'}, {'name': 'James', 'sex': 'M'}]},
 {'class': 'Year 2',
  'student count': 25,
  'room': 'Blue',
  'info': {'teachers': {'physics': 'Albert Einstein'}},
  'students': [{'name': 'Tony', 'sex': 'M'},
   {'name': 'Jacqueline', 'sex': 'F'}]}]

In [57]:
pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']],
)

KeyError: "Try running with errors='ignore' as key 'math' is not always present"

In [59]:
pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']],
    errors='ignore'
)

,name,sex,class,room,info.teachers.math
0,Tom,M,Year 1,Yellow,Rick Scott
1,James,M,Year 1,Yellow,Rick Scott
2,Tony,M,Year 2,Blue,NaN
3,Jacqueline,F,Year 2,Blue,NaN


## Custom separator using sep

In [60]:
data = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 'math': 'Rick Scott', 'physics': 'Elon Mask' }
        },
        'students': [
            { 'name': 'Tom', 'sex': 'M', 'grades': { 'math': 66, 'physics': 77 } },
            { 'name': 'James', 'sex': 'M', 'grades': { 'math': 80, 'physics': 78 } },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 'math': 'Alan Turing', 'physics': 'Albert Einstein' }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

pd.json_normalize(
    data, 
    record_path =['students'], 
    meta=['class', 'room', ['info', 'teachers', 'math']],
    sep='->'
)

,name,sex,grades->math,grades->physics,class,room,info->teachers->math
0,Tom,M,66.0,77.0,Year 1,Yellow,Rick Scott
1,James,M,80.0,78.0,Year 1,Yellow,Rick Scott
2,Tony,M,NaN,NaN,Year 2,Blue,Alan Turing
3,Jacqueline,F,NaN,NaN,Year 2,Blue,Alan Turing


## Adding prefix for meta and record data using meta_prefix and record_prefix

In [63]:
pd.json_normalize(
    data, 
    record_path=['students'], 
    meta=['class'],
    meta_prefix='meta-',
    record_prefix='student-'
)


,student-name,student-sex,student-grades.math,student-grades.physics,meta-class
0,Tom,M,66.0,77.0,Year 1
1,James,M,80.0,78.0,Year 1
2,Tony,M,NaN,NaN,Year 2
3,Jacqueline,F,NaN,NaN,Year 2


## Working with a local file

In [65]:
import json
# load data using Python JSON module
with open('data/simple.json','r') as f:
    data = json.loads(f.read())
    
# Normalizing data
pd.json_normalize(data)

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78


## Working with a URL

In [67]:
import requests

URL = 'http://raw.githubusercontent.com/BindiChen/machine-learning/master/data-analysis/027-pandas-convert-json/data/simple.json'

data = json.loads(requests.get(URL).text)
pd.json_normalize(data)

,id,name,math,physics,chemistry
0,A001,Tom,60,66,61
1,A002,James,89,76,51
2,A003,Jenny,79,90,78


# Pandas Series

## from a Python list

In [70]:
companies = ['Google', 'Microsoft', 'Facebook', 'Apple']

pd.Series(companies)

0       Google
1    Microsoft
2     Facebook
3        Apple
dtype: object

In [74]:
# Custom Index
pd.Series(companies,index=[100,101,102,103])

100       Google
101    Microsoft
102     Facebook
103        Apple
dtype: object

In [76]:
# Custom Index
pd.Series(companies,index=[100,120,150,180])

100       Google
120    Microsoft
150     Facebook
180        Apple
dtype: object

In [78]:
# Custom Index
pd.Series(companies,index=['GOOGL','MSFT','FB','AAPL'])

GOOGL       Google
MSFT     Microsoft
FB        Facebook
AAPL         Apple
dtype: object

In [82]:
# from a dict
companies = {
    'a': 'Google',
    'b': 'Microsoft',
    'c': 'Facebook',
    'd': 'Apple'
}
pd.Series(companies)



a       Google
b    Microsoft
c     Facebook
d        Apple
dtype: object

In [85]:
# When index is specified
pd.Series(
    companies, 
    index=['a', 'b', 'd']
)

a       Google
b    Microsoft
d        Apple
dtype: object

In [87]:
# When index is specified
pd.DataFrame(
    companies, 
    index=['a', 'b', 'd']
)

,a,b,c,d
a,Google,Microsoft,Facebook,Apple
b,Google,Microsoft,Facebook,Apple
d,Google,Microsoft,Facebook,Apple


In [89]:
# from a scalar
pd.Series(10, index=[100, 101, 102, 103])

100    10
101    10
102    10
103    10
dtype: int64

In [93]:
pwd

'C:\\Users\\nemat\\OneDrive - mcw.edu\\DATA_Science\\Python Tutorial'

In [95]:
# with the read_csv()
# By default, the Pandas read_csv() function will import data as a DataFrame.

pd.read_csv('./data/data.csv')


,date,product,price,cost,profit
0,2019/1/1,A,10,5,1
1,2019/1/2,B,20,12,2
2,2019/1/3,C,30,20,3
3,2019/1/4,D,40,30,4


If we want the data to be imported into a Series instead of a DataFrame, we can provide additional arguments usecols and squeeze. The squeeze=True will convert a DataFrame of one column into a Series.



In [98]:
pd.read_csv('./data/data.csv', usecols=['product' , 'cost'], squeeze=True)


,product,cost
0,A,5
1,B,12
2,C,20
3,D,30


## Retrieving data
 with position

In [101]:
s = pd.Series([1,2,3,4,5],index = ['a','b','c','d','e'])

s[0]
s[:3]
s[-3:]
# Retrieving elements within a range
s[2:4]
# Retrieving elements by step
s[::2]

1

a    1
b    2
c    3
dtype: int64

c    3
d    4
e    5
dtype: int64

c    3
d    4
dtype: int64

a    1
c    3
e    5
dtype: int64

In [103]:
# with index/label
s = pd.Series([1,2,3,4,5],index = ['a','b','c','d','e'])
s['a']
s[['b','c','d']]

1

b    2
c    3
d    4
dtype: int64

## Attributes

In [105]:
companies = ['Google', 'Microsoft', 'Facebook', 'Apple']
s = pd.Series(companies)

In [109]:
# The values attribute returns an array of all the values within the series.
s.values

# The index attribute returns a RangeIndex object. 
# We can see it starts at 0 and stops at 5. 
# The last part is called step and that's telling us that it's incrementing by 1
s.index

# The is_unique attribute returns a boolean (True or False). 
# It is a really convenient way to check if every series value is unique or not.
s.is_unique

array(['Google', 'Microsoft', 'Facebook', 'Apple'], dtype=object)

RangeIndex(start=0, stop=4, step=1)

True

## Data type and size

In [111]:
s.dtype
s.size
s.shape
s.ndim

dtype('O')

4

(4,)

1

## Performing aggregations

In [113]:

prices = [10, 5, 3, 2.5, 8, 11]

s = pd.Series(prices)

s.mean()
s.sum()
s.product()


6.583333333333333

39.5

33000.0

In [115]:
s.agg(['mean','sum','product'])


mean           6.583333
sum           39.500000
product    33000.000000
dtype: float64

## Counting values

In [117]:
s = pd.Series(['a','b','b','a','a'])
s.unique()
s.nunique()
s.value_counts()


array(['a', 'b'], dtype=object)

2

a    3
b    2
dtype: int64

## Sorting by values or index labels

In [119]:
prices = [10, 5, 3, 2.5, 8, 11]

s = pd.Series(prices)

# ascending by default
s.sort_values()

3     2.5
2     3.0
1     5.0
4     8.0
0    10.0
5    11.0
dtype: float64

In [121]:
# To sort it in descenting order
s.sort_values(ascending=False)

5    11.0
0    10.0
4     8.0
1     5.0
2     3.0
3     2.5
dtype: float64

In [122]:
# To modify the original series
s.sort_values(inplace=True)
s

3     2.5
2     3.0
1     5.0
4     8.0
0    10.0
5    11.0
dtype: float64

In [123]:
# ascending by default
s.sort_index()

0    10.0
1     5.0
2     3.0
3     2.5
4     8.0
5    11.0
dtype: float64

In [125]:
# To sort it in descenting order
s.sort_index(ascending=False)

5    11.0
4     8.0
3     2.5
2     3.0
1     5.0
0    10.0
dtype: float64

In [126]:
# To modify the original series
s.sort_index(inplace=True)

In [128]:
s

0    10.0
1     5.0
2     3.0
3     2.5
4     8.0
5    11.0
dtype: float64

## Working with missing values

In [143]:
s = pd.Series([1, 2, 3,np.nan,  10, 4 , 5 , np.nan])
s.isna()

0    False
1    False
2    False
3     True
4    False
5    False
6    False
7     True
dtype: bool

In [142]:
s.isna().sum()
s.count()

2

6

## Searching values

In [146]:
prices = [10, 5, 3, 2.5, 8, 11]

s = pd.Series(prices)
s.nlargest()
s.nlargest(2)

5    11.0
0    10.0
4     8.0
1     5.0
2     3.0
dtype: float64

5    11.0
0    10.0
dtype: float64

In [148]:
s.le(5, fill_value=0)
s <= 5


0    False
1     True
2     True
3     True
4    False
5    False
dtype: bool

0    False
1     True
2     True
3     True
4    False
5    False
dtype: bool

In [152]:
prices = [10, 5, 3, 2.5, 8, 11]

s = pd.Series(prices)

len(s)
type(s)

6

pandas.core.series.Series

In [154]:
list(s)

[10.0, 5.0, 3.0, 2.5, 8.0, 11.0]

In [156]:
dict(s)

{0: 10.0, 1: 5.0, 2: 3.0, 3: 2.5, 4: 8.0, 5: 11.0}

In [158]:
# by default Pandas is going to look among the index labels not the actual values within the Series. 
2.5 in s

False

In [160]:
s.values

array([10. ,  5. ,  3. ,  2.5,  8. , 11. ])

In [159]:
2.5 in s.values

True